<a href="https://colab.research.google.com/github/aminaalisheva/Unigrams-and-Sentiment-Classification-on-Kazakh-dataset/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/3-year SEDS/NLP_benchmark/ass2/dataset.csv"
df = pd.read_csv(file_path, encoding="utf-8")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,url,retrieval_date
0,Қазақ халқының ұлттық ертегілері\n\nЖолбарыс б...,https://ertegiler.kz/story/qu-tulki,2025-01-29
1,Қазақ халқының ұлттық ертегілері\n\nЕрте заман...,https://ertegiler.kz/story/sabige-omir-syilaga...,2025-01-29
2,Ханс Кристиан Андерсен\n\nМенің бейшара гүлдер...,https://ertegiler.kz/story/kishkene-idanyn-gul...,2025-01-29
3,Басқа\n\nБұрынғы өткен заманда үш торай өмір с...,https://ertegiler.kz/story/ush-torai,2025-01-29
4,Қазақ халқының ұлттық ертегілері\n\nБір күні Қ...,https://ertegiler.kz/story/kozhanasyr-men-khan,2025-01-29


In [ ]:
import re
from collections import Counter
from nltk.util import ngrams

# Use your Kazakh-specific tokenizer
kazakh_pattern = r'\b[\wәғқңүұөһі-]+\b'
tokens = []
text_data = df['text']

# Tokenize using your method
for text in text_data.dropna():
    text = re.sub(r'(\b\w+)-(\b\w+)\b',
    lambda m: m.group(1) if m.group(1) == m.group(2) else m.group(0), # Merge duplicate hyphenated words
    text.lower())
    tokens.extend(re.findall(kazakh_pattern, text))

# Token stats
print(f"Total Tokens: {len(tokens)}")
print(f"Unique Words (Types): {len(set(tokens))}")
print("Top 10 Word Frequencies:", Counter(tokens).most_common(10))

Total Tokens: 128510
Unique Words (Types): 23030
Top 10 Word Frequencies: [('бір', 1780), ('деп', 1694), ('мен', 1272), ('да', 1110), ('дейді', 1063), ('ол', 984), ('де', 886), ('екен', 835), ('алып', 686), ('бұл', 671)]


In [ ]:
unigrams = tokens
bigrams = list(ngrams(tokens, 2)) #lists to count them
trigrams = list(ngrams(tokens, 3))

unigram_freq = Counter(unigrams)
bigram_freq = Counter(bigrams)
trigram_freq = Counter(trigrams)

vocab_size = len(set(tokens))

print("Top 5 Unigrams:", unigram_freq.most_common(5))
print("Top 5 Bigrams:", bigram_freq.most_common(5))
print("Top 5 Trigrams:", trigram_freq.most_common(5))

Top 5 Unigrams: [('бір', 1780), ('деп', 1694), ('мен', 1272), ('да', 1110), ('дейді', 1063)]
Top 5 Bigrams: [(('бір', 'күні'), 199), (('деп', 'сұрайды'), 119), (('тағы', 'да'), 107), (('қазақ', 'халқының'), 82), (('халқының', 'ұлттық'), 81)]
Top 5 Trigrams: [(('қазақ', 'халқының', 'ұлттық'), 81), (('халқының', 'ұлттық', 'ертегілері'), 81), (('деп', 'жауап', 'береді'), 50), (('шал', 'мен', 'кемпір'), 20), (('екен', 'бір', 'күні'), 19)]


In [ ]:
import math

def calc_perplx(ngram_freq, lower_order_freq=None):
    total_count = sum(ngram_freq.values())
    perplexity = 0

    for ngram, count in ngram_freq.items():
        if lower_order_freq:  # Bigram or Trigram
            prefix = ngram[:-1]
            prefix_count = lower_order_freq[prefix]
            prob = count / prefix_count if prefix_count > 0 else 1e-10
        else:  # Unigram
            prob = count / total_count
        perplexity += count * math.log(prob, 2)  # log probability

    perplexity = 2 ** (-perplexity / total_count)
    return perplexity

uni_perplx = calc_perplx(unigram_freq)
bi_perplx = calc_perplx(bigram_freq, Counter(ngrams(tokens, 1)))
tri_perplx = calc_perplx(trigram_freq, Counter(ngrams(tokens, 2)))

print(f"Unigram Perplexity: {uni_perplx}")
print(f"Bigram Perplexity: {bi_perplx}")
print(f"Trigram Perplexity: {tri_perplx}")

Unigram Perplexity: 4247.374575328174
Bigram Perplexity: 18.792801079557137
Trigram Perplexity: 1.4652946183990276


In [ ]:
import math

def calculate_perplexity(prob_func, ngrams_list):
    log_prob_sum = 0
    N = len(ngrams_list)

    for ngram in ngrams_list:
        prob = prob_func(ngram)
        prob = max(prob, 1e-10)  # Prevent log(0)
        log_prob_sum += math.log(prob, 2)

    return 2 ** (-log_prob_sum / N)


In [ ]:
#TASK2
def laplace_prob(ngram, ngram_freq, lower_order_freq, vocab_size):
    if len(ngram) == 1:  # Unigram probability
        return (ngram_freq.get(ngram[0], 0) + 1) / (sum(ngram_freq.values()) + vocab_size)
    else:  # Bigram or Trigram probability
        prefix = ngram[:-1]
        prefix_count = lower_order_freq.get(prefix, 0)
        return (ngram_freq.get(ngram, 0) + 1) / (prefix_count + vocab_size)


In [ ]:
def interpolated_prob(ngram, lambdas=(0.1, 0.3, 0.6)):
    unigram = (ngram[-1],)
    bigram = ngram[-2:] if len(ngram) >= 2 else None
    trigram = ngram if len(ngram) == 3 else None

    # Unigram probability
    p1 = unigram_freq.get(unigram[0], 0) / sum(unigram_freq.values())

    # Bigram probability
    p2 = bigram_freq.get(bigram, 0) / unigram_freq.get(bigram[0], 1) if bigram else 0

    # Trigram probability
    p3 = trigram_freq.get(trigram, 0) / bigram_freq.get(trigram[:-1], 1) if trigram else 0

    return lambdas[0] * p1 + lambdas[1] * p2 + lambdas[2] * p3


In [ ]:
def backoff_prob(ngram, alpha=0.4):
    if len(ngram) == 3 and trigram_freq.get(ngram, 0) > 0:
        return trigram_freq[ngram] / bigram_freq.get(ngram[:-1], 1)
    elif len(ngram) >= 2 and bigram_freq.get(ngram[-2:], 0) > 0:
        return alpha * (bigram_freq[ngram[-2:]] / unigram_freq.get(ngram[-2], 1))
    else:
        return alpha * (unigram_freq.get(ngram[-1], 0) / sum(unigram_freq.values()))


In [ ]:
def kneser_ney_prob(ngram, discount=0.75):
    if len(ngram) == 1:
        return unigram_freq.get(ngram[0], 0) / sum(unigram_freq.values())

    prefix = ngram[:-1]
    prefix_count = bigram_freq.get(prefix, 1)
    continuation_count = len([ng for ng in bigram_freq if ng[1] == ngram[-1]])

    return max((trigram_freq.get(ngram, 0) - discount), 0) / prefix_count + \
           (discount * continuation_count / prefix_count) * \
           (unigram_freq.get(ngram[-1], 0) / sum(unigram_freq.values()))


In [ ]:
# Laplace Perplexity
laplace_perplexity = calculate_perplexity(lambda ng: laplace_prob(ng, trigram_freq, bigram_freq, vocab_size), trigrams)

# Interpolation Perplexity
interpolation_perplexity = calculate_perplexity(interpolated_prob, trigrams)

# Backoff Perplexity
backoff_perplexity = calculate_perplexity(backoff_prob, trigrams)

# Kneser-Ney Perplexity
kneser_ney_perplexity = calculate_perplexity(lambda ng: kneser_ney_prob(ng), trigrams)

# Print Results
print(f"Laplace Perplexity: {laplace_perplexity}")
print(f"Interpolation Perplexity: {interpolation_perplexity}")
print(f"Backoff Perplexity: {backoff_perplexity}")
print(f"Kneser-Ney Perplexity: {kneser_ney_perplexity}")


Laplace Perplexity: 10841.133392717926
Interpolation Perplexity: 2.188910834156973
Backoff Perplexity: 1.4652946183990259
Kneser-Ney Perplexity: 3.676953817893042
